<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data Solutions

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [11]:
import csv
import pandas as pd
from collections import namedtuple   # Convenient to store the data rows

DATA_FILE = 'chipotle.tsv'

In [36]:
chipotle_csv = pd.read_csv(DATA_FILE, sep='\t', header=None)
chipotle_csv.head()

,0,1,2,3,4
0,order_id,quantity,item_name,choice_description,item_price
1,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
2,1,1,Izze,[Clementine],$3.39
3,1,1,Nantucket Nectar,[Apple],$3.39
4,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39


In [17]:
# Specify that the delimiter is a tab character
with open(DATA_FILE, mode='r') as f:
    chipotle_csv = [row for row in csv.reader(f, delimiter='\t')]

### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [37]:
header = chipotle_csv[:1]
header.head()

,0,1,2,3,4
0,order_id,quantity,item_name,choice_description,item_price


In [35]:
ChipotleData = namedtuple("ChipotleData", header)

data = [ChipotleData._make(x) for x in chipotle_csv[1:]]
data

ValueError: Type names and field names must be valid identifiers: '0'

---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [23]:
# Count the number of unique order_ids
# Note: You could assume this is 1,834 as that's the maximum order_id, but it's best to check
num_orders = len(set(row.order_id for row in data))

num_orders

1834

In [34]:
print(data[0].item_price)

# We must convert the price from a string to a float ...
print(data[0].item_price[1:-1])  # Strip the dollar sign and trailing space

# Alternatively (a bit more flexible) ...
print(data[0].item_price.strip(' $'))

$2.39 
2.39
2.39


In [38]:
# Create a list of prices
# Note: Ignore the 'quantity' column because the 'item_price' takes quantity into account
prices = [float(row.item_price.strip(' $')) for row in data]

In [39]:
# Calculate the average price of an order and round to two digits
round(sum(prices) / num_orders, 2)      # $18.81

18.81

### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [43]:
# For each canned drink, get the choice description
#  (Note this assumes there is exactly one descr in the brackets!)
sodas = []
for row in data:
    if 'Canned' in row.item_name:
        sodas.append(row.choice_description.strip('[]'))

# Example
sodas[:5]

['Sprite', 'Dr. Pepper', 'Mountain Dew', 'Sprite', 'Dr. Pepper']

In [44]:
# Equivalent list comprehension (using an 'if' condition)
sodas = [row.choice_description.strip('[]') \
         for row in data if 'Canned' in row.item_name]

In [45]:
# Create a set of unique sodas
unique_sodas = set(sodas)

In [46]:
unique_sodas

{'Coca Cola',
 'Coke',
 'Diet Coke',
 'Diet Dr. Pepper',
 'Dr. Pepper',
 'Lemonade',
 'Mountain Dew',
 'Nestea',
 'Sprite'}

---

## Advanced Level

### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


In [47]:
# Keep a running total of burritos and toppings
burrito_count = 0
topping_count = 0

In [48]:
# Calculate the number of toppings by counting the commas and adding 1
# Note: x += 1 is equivalent to x = x + 1
for row in data:
    if 'Burrito' in row.item_name:
        burrito_count += 1
        topping_count += (row.choice_description.count(',') + 1)

In [49]:
# Calculate the average topping count and round to two digits
avg_toppings = round(topping_count / float(burrito_count), 2)

In [50]:
avg_toppings

5.4

### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

In [51]:
# Start with an empty dictionary
chips = {}


In [53]:
# If chip order is not in a dictionary, then add a new key/value pair
# If chip order is already in a dictionary, then update the value for that key
for row in data:
    if 'Chips' in row.item_name:
        if row.item_name not in chips:
            # This is a new key, so create a key/value pair
            chips[row.item_name] = int(row.quantity)
        else:
            # This is an existing key, so it add to the value
            chips[row.item_name] += int(row.quantity)

In [54]:
chips

{'Chips': 230,
 'Chips and Fresh Tomato Salsa': 130,
 'Chips and Guacamole': 506,
 'Chips and Mild Fresh Tomato Salsa': 1,
 'Chips and Roasted Chili Corn Salsa': 23,
 'Chips and Roasted Chili-Corn Salsa': 18,
 'Chips and Tomatillo Green Chili Salsa': 45,
 'Chips and Tomatillo Red Chili Salsa': 50,
 'Chips and Tomatillo-Green Chili Salsa': 33,
 'Chips and Tomatillo-Red Chili Salsa': 25,
 'Side of Chips': 110}

In [55]:
# .defaultdict() saves you the trouble of checking whether or not a key already exists
from collections import defaultdict

dchips = defaultdict(int)
for row in data:
    if 'Chips' in row.item_name:
        dchips[row.item_name] += int(row.quantity)

In [56]:
dchips

defaultdict(int,
            {'Chips': 230,
             'Chips and Fresh Tomato Salsa': 130,
             'Chips and Guacamole': 506,
             'Chips and Mild Fresh Tomato Salsa': 1,
             'Chips and Roasted Chili Corn Salsa': 23,
             'Chips and Roasted Chili-Corn Salsa': 18,
             'Chips and Tomatillo Green Chili Salsa': 45,
             'Chips and Tomatillo Red Chili Salsa': 50,
             'Chips and Tomatillo-Green Chili Salsa': 33,
             'Chips and Tomatillo-Red Chili Salsa': 25,
             'Side of Chips': 110})

---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!
